In [1]:
import pandas as pd
import bs4 as bs
import re
import requests
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
from time import sleep
from os import listdir
from os.path import isfile, join, isdir
import random
import itertools
from pywebcopy import save_webpage

## Generate HTML Test Panels

In [2]:
test_links = pd.read_csv('./test_links.csv')

In [3]:
# get all files from path
mypath = '/Users/jeloretizo/Documents/MSBA/Practicum Project/COPA-Usability-Testing'
files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and ('ratio' in f)]

In [10]:
query_strings = ['brakes during taxi',
                 'capture glide slope above',
                 'stall under hood',
                 'find flight instructor',
                 'oxygen altitude',
                 'file nasa report',
                 'loose fairing noise',
                 'loose seat belt',
                 'power off landing',
                 'music volume low']

In [5]:
panel_labels = ['Panel1.html', 'Panel2.html', 'Panel3.html', 'Panel4.html']

In [31]:
def get_rerank_list(soup, rerank):
    ''' Helper function to rerank soup files'''
    all_items = soup.find_all(class_='fps-result')
    html_links = pd.DataFrame(all_items).reset_index()
    html_links = html_links.rename(columns={'index':'old_rank'})
    html_links['id'] = html_links[3].apply(lambda x: get_href(x))
    rerank = rerank.reset_index(drop=True).reset_index()
    rerank = rerank.rename(columns={'index': 'New Rank'})
    for i in html_links['id']:
        try: 
#             print(rerank.loc[temp_var['url-id'].str.contains(i)])
            new_rank = rerank.loc[rerank['url-id'].str.contains(i)].index[0]
        except:
            new_rank = None
        html_links.loc[html_links['id'] == i, 'New Rank'] = new_rank
#     print(html_links.head())
    new_order = list(html_links.sort_values('New Rank', ascending=True)['old_rank'])
#     print(html_links.head)
    final_order = [all_items[i] for i in new_order]
    return final_order

In [7]:
def get_id(item):
    ''' Helper function to get id from ratio combo file '''
    res_url = re.findall(r'\/[0-9]+\/[0-9]+|\/[\d]+$',item)[0]
    return res_url

In [8]:
def get_href(html):
    ''' Helper function to get id from the html files '''
    href = html.find(class_='search-link')['href']
    return get_id(href)

In [33]:
for i in tqdm(query_strings):
#     print(i)
    random_panels = random.sample(panel_labels, 4)
    folder_name = i.title().replace(' ', '')
    query_path = mypath + '/' + folder_name + '/'
    file = [f for f in listdir(query_path) if (
        i in f) and (isdir(join(query_path, f)) == False)]
    if len(file) == 1:
        with open('./{}/{}'.format(folder_name, file[0]), 'r') as open_file:
            soup = bs.BeautifulSoup(open_file, 'html.parser')
            saved_soup = soup
            print(i, ' -- Control Panel Printed in:', random_panels[3])
            with open('./{}/{}'.format(folder_name, random_panels[3]), 'w') as output_file:
                test_links.loc[(test_links['0'] == folder_name) & (test_links['1'] == random_panels[3]), 
                              'Panel Type'] = 'Control'
                output_file.write(str(saved_soup))
                output_file.close()
            model_number = 0
            for j in files:
                ratio = pd.read_csv('./' + j)
                ratio['url-id'] = ratio['url'].apply(lambda x: get_id(x))
                temp_var = ratio.loc[ratio['Query'] == i]
                final_list = get_rerank_list(soup, temp_var)
#                 print(final_list)
                with open('./{}/{}'.format(folder_name, file[0]), 'r') as open_file:
                    soup = bs.BeautifulSoup(open_file, 'html.parser')
                for k in range(len(final_list)):
                    soup.find_all(
                        class_='fps-result')[k].replace_with(final_list[k])
                print(i, ' -- Model Ratio: ', j, ' -- Printed in: ',
                      random_panels[model_number])
                test_links.loc[(test_links['0'] == folder_name) & (test_links['1'] == random_panels[model_number]), 
                              'Panel Type'] = j
                with open('./{}/{}'.format(folder_name, random_panels[model_number]), 'w') as output_file:
                    output_file.write(str(soup))
                    output_file.close()
                model_number += 1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


brakes during taxi  -- Control Panel Printed in: Panel1.html
brakes during taxi  -- Model Ratio:  ratio_comb3.csv  -- Printed in:  Panel2.html
brakes during taxi  -- Model Ratio:  ratio_comb2.csv  -- Printed in:  Panel3.html
brakes during taxi  -- Model Ratio:  ratio_comb1.csv  -- Printed in:  Panel4.html
capture glide slope above  -- Control Panel Printed in: Panel2.html
capture glide slope above  -- Model Ratio:  ratio_comb3.csv  -- Printed in:  Panel3.html
capture glide slope above  -- Model Ratio:  ratio_comb2.csv  -- Printed in:  Panel4.html
capture glide slope above  -- Model Ratio:  ratio_comb1.csv  -- Printed in:  Panel1.html
stall under hood  -- Control Panel Printed in: Panel2.html
stall under hood  -- Model Ratio:  ratio_comb3.csv  -- Printed in:  Panel1.html
stall under hood  -- Model Ratio:  ratio_comb2.csv  -- Printed in:  Panel3.html
stall under hood  -- Model Ratio:  ratio_comb1.csv  -- Printed in:  Panel4.html
find flight instructor  -- Control Panel Printed in: Panel3

In [34]:
test_links.to_csv('./test_links.csv', index=False)